# RAG Local con Ollama y LangChain
Este notebook implementa un sistema RAG (Retrieval-Augmented Generation) completamente local usando:
- `Ollama` para correr modelos de lenguaje y de embeddings
- `LangChain` para orquestar el flujo RAG
- `FAISS` como motor de vector store local
- Archivos locales como fuente de información

Vamos paso a paso.

In [ ]:
# Instala las dependencias necesarias (solo la primera vez)
!pip install langchain faiss-cpu sentence-transformers openai chromadb
# Nota: asume que Ollama ya está instalado y configurado localmente

## Paso 1: Carga y preprocesamiento de documentos
Vamos a cargar algunos archivos `.txt` desde una carpeta local llamada `docs/`.
Luego, los dividimos en fragmentos (chunks) para alimentar al sistema de embeddings.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

docs_path = Path("docs")
loaders = [TextLoader(str(f)) for f in docs_path.glob("*.txt")]
documents = []
for loader in loaders:
    documents.extend(loader.load())

# Split en chunks de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)
print(f"Documentos cargados: {len(documents)}, Fragmentos generados: {len(split_docs)}")

## Paso 2: Generación de Embeddings con Ollama
Usamos un modelo local de Ollama para crear los vectores de cada chunk.
Requiere que hayas descargado el modelo de embeddings, como `nomic-embed-text`.

In [ ]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

embedding = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
# Guardamos en disco para futuras ejecuciones
vectorstore.save_local("faiss_index")

## Paso 3: Consulta y generación de respuesta con contexto
Le hacemos una pregunta al sistema. Usamos el vector store para buscar los fragmentos más relevantes
y luego un modelo como `llama3` para generar la respuesta basada en esos fragmentos.

In [ ]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

# Cargamos el índice
vectorstore = FAISS.load_local("faiss_index", embedding)
retriever = vectorstore.as_retriever()
llm = Ollama(model="llama3")
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

question = "¿Cuál es el concepto principal en los documentos cargados?"
respuesta = qa_chain.run(question)
print("Respuesta generada:", respuesta)

## Conclusión
Este flujo demuestra cómo implementar RAG de forma **100% local** usando Ollama, LangChain, FAISS y archivos de texto.
Puedes expandir esto integrando PDF, CSV, herramientas de UI o conectores con otras apps como Notion, n8n, etc.